<a href="https://colab.research.google.com/github/ShlokArora2709/CogitoQuotient/blob/main/model_training/Dress_model_taining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'cloth-class-data:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4487680%2F7690069%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240228%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240228T061259Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D1f1ad02ae26a629b1c788393a70a3d04c27121be572a2536e57ede3241633c2e02df534985025b00819a73b4c881ea4a0ed74ea84102119b5b21810b09ee763e9fd391ec2bedc5af2af27ca2afd871184a05ab70d37bcaa7c2a4a544bfc47732de583c7b62f4ba9608c010961b8ad857abe30f5dcccd13e92a1c904a80950bd0de81654356dd23ae7e5992d25695692519e2a300ab58263786dc555f80c63a4be046a6003af2e83f5aada38e4115e0775f6a85f1776e67ef9eff05f8021e1a2ddf55ea8e96e648af12a8d45b3227c3bb734032b295fb239a800119e13a1cbd239e0c48f0064f76963d1b02533469c645d3626b11a69df8e7787dcce014774fe7'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
import numpy as np
from keras.applications import VGG16,InceptionResNetV2
from keras.models import Model
from keras.layers import LSTM, Dense, Dropout, Flatten
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
import matplotlib.pyplot as plt

import os, random, pathlib, warnings, itertools, math
warnings.filterwarnings("ignore")

import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.backend as K

from keras import regularizers
from tensorflow.keras import models,Sequential,layers
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.inception_v3 import InceptionV3,preprocess_input
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, Dense, Dropout,LeakyReLU
from tensorflow.keras.optimizers import Adam
from keras.applications.vgg16 import VGG16
import cv2

In [ ]:
IMAGE_SIZE=[256,256]
train_datagen = image.ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=40,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest')

test_datagen = image.ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory(
    '/kaggle/input/cloth-class-data/data_cloth/train',
    target_size = IMAGE_SIZE,
    batch_size = 64,
    class_mode = 'categorical')

test_set = test_datagen.flow_from_directory(
    '/kaggle/input/cloth-class-data/data_cloth/test',
    target_size = IMAGE_SIZE,
    batch_size = 64,
    class_mode = 'categorical')


In [ ]:
test_set.class_indices

In [ ]:
training_set.class_indices

In [ ]:
base= InceptionResNetV2(weights='imagenet', include_top=False)

for layer in base.layers:
    layer.trainable = True
    if isinstance(layer, tf.keras.layers.Conv2D):
        layer.kernel_regularizer = kernel_regularizer=regularizers.L2(0.005)

# Find the index of the layer where 'mixed9' is present
block8_1_mixed = base.layers.index(next(layer for layer in base.layers if layer.name == 'block8_1_mixed'))

# Set trainable to False for layers before 'mixed9'
for layer in base.layers[:block8_1_mixed]:
    layer.trainable = False

for layer in base.layers:
    print(layer.name, layer.trainable)

In [ ]:
model = Sequential()
model.add(base)
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.5))
model.add(Dense(1024,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(6,activation='softmax'))

model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)
model.summary()

In [ ]:
his = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=10)

In [ ]:
pred=model.predict_generator(test_set)
y_pred = np.argmax(pred, axis=1)
y_true = test_set.classes

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

conf_matrix = confusion_matrix(y_true, y_pred)

# Plot heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='YlOrBr')

# Add labels and title
plt.xlabel('Predicted labels')
plt.ylabel('Actual labels')
plt.title('Confusion Matrix')

# Display the heatmap
plt.show()

In [ ]:
category={0:'0coat',
 1:'1shirt',
 2:'Daily women',
 3:'Informal shirts',
 4:'Kurta',
 5:'Tshirt'}

In [ ]:
def predict_image(filename,model):
    img_ = image.load_img(filename, target_size=IMAGE_SIZE)
    img_array = image.img_to_array(img_)
    img_processed = np.expand_dims(img_array, axis=0)
    img_processed /= 255.

    prediction = model.predict(img_processed)
    index = np.argmax(prediction)

    plt.title("Prediction - {}".format(category[index]))
    plt.imshow(img_array)

In [ ]:
predict_image('/kaggle/input/cloth-class-data/data_cloth/test/Informal shirts/5372.jpg',model)

In [ ]:
model.save("Clothing_91.h5")